In [2]:
#importing python modules
import numpy as np
from scipy.linalg import hilbert
from sympy import Matrix
import math
from fractions import Fraction

In [3]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)


def signif(x, p):
    x = np.asarray(x)
    x_positive = np.where(np.isfinite(x) & (x != 0), np.abs(x), 10**(p-1))
    mags = 10 ** (p - 1 - np.floor(np.log10(x_positive)))
    return np.round(x * mags) / mags


In [4]:
#Function to create a nxn hilbert matrix
def Hilbert(n):
    H = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            H[i][j] = 1/(float(i + j + 1))
    H = np.around(H, decimals=4)
    return H

#Actual solution vector
def X(n):
    x = np.ones([n, 1])
    return x

#vector b, b = H * x
def B(H, x):
    b = np.dot(H, x)
    b = Matrix(b)
    return b


In [5]:
#Gaussian Elimination with no pivoting: Forward Elimination and Backwards Substitution
def Gaussian_Elimination(H, b):
    #Find n, the number of rows of the hilbert matrix
    n = H.shape[0]
    total_mult = 0
    #Step 1: Check for Errors
    if any(np.diag(H) == 0):
        raise ZeroDivisionError(
            ('Division by zero will occur; ''pivoting currently not supported'))
    #Initialize an array x of size n rows, 1 column
    x = np.zeros([n, 1])

    #row = first row
    for row in range(n):

        #Forward Substitution
        for i in range(row + 1, n):  # i = the row below the 1st pivot row
            #calculating the multiplier
            #Textbook: mji = Hji/Hii
            #if row = 0 then i = 1
            #multiplier[0, 1] = H[0,1]/H[0,0]
            multiplier = H[row, i]/H[row, row]
            total_mult = total_mult + 1
            #Calculating the new b element for i
            # new b value for row i = old b value for row i minus the multiplier times the b value for "row"
            b[i] = b[i] - multiplier * b[row]
            total_mult = total_mult + 1
            for j in range(row + 1, n):
                #Compute the new values of H[i,j]
                #Recall: H[0, 1] = first row
                #Calculating the new second row through Gaussian Elim:
                #second row = second row - multiplier times the initial row
                H[i, j] = H[i, j] - multiplier * H[row, j]
                total_mult = total_mult + 1

    #Solving for the x values using backwards substitution
    #Given the new updated Hx = b matrix
    #Starting from the last row and backtracking
    #range(start, end, step)
    for j in range(n-1, -1, -1):
        # x = b value/H value
        #Example: solving for the first x value: x[5] = b[5]/H[5,5], x5 = last b element divided by H element at row 5, col 5
        #update the x matrix and put the x[j] value into the last row
        #solving for the last x value first, x[last] = b[last]/H[last row,last column]
        x[j] = b[j]/H[j, j]
        x[j] = trunc(x[j], 4)
        total_mult = total_mult + 1
        #ex n =5, b[4] = b[4] - H[4,5] * x[5], already obtained x value for last column
        for i in range(j):
            b[i] = b[i] - H[i, j] * x[j]
            total_mult = total_mult + 1

    #return computed solution vector and total multiplications performed
    return x, total_mult


In [6]:
#computing the error, error = Actual solution vector - Computed solution vector
def Error(x, xx):
    error = x - xx
    error = np.array(error)
    error = signif(error, 4)
    return error

#Infinity Norm of the error vector is the element in the error vector with the max absolute value
def Infinity_Norm(error_vector):
    inf_norm = np.linalg.norm(error_vector, np.inf)
    inf_norm = round(inf_norm, 4)
    return inf_norm

#Euclidean norm is the magnitude of the error vector, same as distance formula
def Euclidean_Norm(error_vector):
    n = error_vector.shape[0]
    euc = 0
    sum = 0
    for i in range(n):
        sum += error_vector[i] ** 2
    euc = math.sqrt(sum)
    euc = round(euc, 4)
    return euc


#Expected total number of multiplications according to the book
def formula_mult(n):
    total = ((n**3)/3) + (n**2) - (n/3)
    total = round(total, 0)
    return total


In [7]:
#Main Program
if __name__ == '__main__':
    n = [11, 12, 13]
    for i in n:
        H = Hilbert(i)
        x = X(i)
        b = B(H, x)
        xx, total_mult = Gaussian_Elimination(H, b)
        error = Error(x, xx)
        inf_norm = Infinity_Norm(error)
        euc_norm = Euclidean_Norm(error)
        expected_mult = formula_mult(i)
        print("For n =", i)
        print("1. The exact solution is: \n", x)
        print("2. The computed solution is: \n", xx)
        print("3. The error vector is: \n", error)
        print("5. The Euclidean Norm of the error vector is: ", euc_norm)
        print("6. The Number of multiplications in my computer program is: ", total_mult)
        print("7. The number of multiplications using the formula from the book, my answer should have been: ", expected_mult)
        print("--------------------------------------------------------------------------------------------------------------")


For n = 11
1. The exact solution is: 
 [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
2. The computed solution is: 
 [[1.    ]
 [1.    ]
 [0.9999]
 [0.9999]
 [1.0002]
 [0.9998]
 [1.0002]
 [1.    ]
 [0.9999]
 [1.    ]
 [1.    ]]
3. The error vector is: 
 [[ 0.    ]
 [ 0.    ]
 [ 0.0001]
 [ 0.0001]
 [-0.0002]
 [ 0.0002]
 [-0.0002]
 [ 0.    ]
 [ 0.0001]
 [ 0.    ]
 [ 0.    ]]
5. The Euclidean Norm of the error vector is:  0.0004
6. The Number of multiplications in my computer program is:  561
7. The number of multiplications using the formula from the book, my answer should have been:  561.0
--------------------------------------------------------------------------------------------------------------
For n = 12
1. The exact solution is: 
 [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
2. The computed solution is: 
 [[1.    ]
 [0.9998]
 [1.0007]
 [0.9992]
 [0.9999]
 [1.0003]
 [0.9995]
 [1.0005]
 [1.0005]
 [1.0002]
 [0.9994]
 [0.9999]]
3. The er